In [1]:
import pandas as pd
df = pd.read_csv("data_tms.csv")

In [2]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [30]:
def survival_probability(row):

    probability = 1;

    # расчёт вероятности выжить в зависимости от пола
    if row['Sex'] == 'male':
        probability *= 0.19
    else:
        probability *= 0.74

    # расчёт вероятности выжить в зависимости от класса билета
    if row['Pclass'] == 1:
        probability *= 0.63
    elif row['Pclass'] == 2:
        probability *= 0.47
    else:
        probability *= 0.24

    # расчёт вероятности выжить в зависимости от порта посадки
    if row['Embarked'] == 'C':
        probability *= 0.55
    elif row['Embarked'] == 'Q':
        probability *= 0.39
    elif row['Embarked'] == 'S':
        probability *= 0.34

    # расчёт вероятности выжить в зависимости от возрастной группы
    if row['Age'] < 18:
        probability *= 0.54
    elif row['Age'] >= 60:
        probability *= 0.27
    elif pd.isnull(row['Age']):
        probability *= 1
    else:
        probability *= 0.39

    # расчёт вероятности выжить в зависимости от стоимости билета
    if row['Fare'] <= 10:
        probability *= 0.21
    elif row['Fare'] <= 20:
        probability *= 0.42
    elif row['Fare'] <= 30:
        probability *= 0.44
    elif row['Fare'] <= 50:
        probability *= 0.36
    elif row['Fare'] <= 100:
        probability *= 0.65
    else:
        probability *= 0.74

    # расчёт вероятности выжить в зависимости от семьи на борту
    if row['SibSp'] > 0 and row['Parch'] == 0:
        probability *= 0.496
    elif row['Parch'] > 0 and row['SibSp'] == 0:
        probability *= 0.66
    elif row['SibSp'] > 0 and row['Parch'] > 0:
        probability *= 0.44
    else:
        probability *= 0.3

    return probability

data = df.iloc[516]
individual_prob = survival_probability(data)

# Нормализуем итоговую вероятность
average_prob = df.apply(survival_probability, axis = 1).mean()
relative_prob = individual_prob / average_prob

if data['Survived'] == 0:
    fact_survival = 'не выжил.'
else:
    fact_survival = 'выжил.'

# Интерпретируем нормализованную вероятность
if relative_prob > 1:
    prediction = 'высокая'
elif relative_prob < 1:
    prediction = 'низкая'
else:
    prediction = '50/50'
    
print(f'Вероятность выжить: {prediction}. Фактическая выжимаемость: {fact_survival}')

Вероятность выжить: низкая. Фактическая выжимаемость: выжил.


In [34]:
# Вычислим процент правильных ответов
df['IndividualProb'] = df.apply(survival_probability, axis=1)

average_prob = df['IndividualProb'].mean()

df['RelativeProb'] = df['IndividualProb'] / average_prob

df['PredictedSurvived'] = (df['RelativeProb'] > 1).astype(int)

correct_predictions = (df['PredictedSurvived'] == df['Survived']).sum()
total_predictions = len(df)
accuracy = correct_predictions / total_predictions

print(f'Модель правильно предсказала {correct_predictions} из {total_predictions} случаев.')
print(f'Точность модели: {round(accuracy * 100, 2)} %.')

Модель правильно предсказала 676 из 891 случаев.
Точность модели: 75.87 %.
